<a href="https://colab.research.google.com/github/mloyorev/Theory/blob/main/7_OptimalSavingsNumba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install quantecon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 2.1 MB/s eta 0:00:00


In [3]:
from collections import namedtuple
import numpy as np
import quantecon as qe
from numba import njit, prange, int32
import matplotlib.pyplot as plt

Once **again** we use the functions:


*   argmax
*   succesive_approx



In [4]:
@njit
def argmax(list_object):
    max_val = -np.inf
    argmax_index = None
    for i, x in enumerate(list_object):
        if x > max_val:
            max_val = x
            argmax_index = i
    return argmax_index

In [5]:
def successive_approx(T,
                      x_0,
                      tolerance=1e-6,
                      max_iter=10_000,
                      print_step=25,
                      verbose=False):
    x = x_0
    error = tolerance + 1
    k = 1
    while error > tolerance and k <= max_iter:
        x_new = T(x)
        error = np.max(np.abs(x_new - x))
        if verbose and k % print_step == 0:
            print(f"Completed iteration {k} with error {error}.")
        x = x_new
        k += 1
    if error > tolerance:
        print(f"Warning: Iteration hit upper bound {max_iter}.")
    elif verbose:
        print(f"Terminated successfully in {k} iterations.")
    return x


# **Optimal savings model**

We assume that **wealth $W_{t}$ evolves** according to

$$C_{t}+W_{t+1}≤RW_{t}+Y_{t}$$

where $R$ is the gross interest rate, $Y_{t}$ is the labor income and:


*   $(W_{t})$ takes values in a finite set $W$ contained in the set $\mathbb{R}_{+}$
*   $(Y_{t})$ is a **$Q$-Markov chain** on a finite and continuous set $Y$.
*   $C_{t}\geq 0$ for all $t$.

The individual **aims to maximize the present value** of his utility function. Additionally, we suppose that the individual perceive utility from consumption. This means that the consumer's maximization problem is

$$V(w_{t},y_{t})=\max_{\left\{w_{t+\tau+1}\right\}}E_{t}\sum_{\tau=0}^{\infty}\beta^{\tau}u(c_{t})=\max_{\left\{w_{t+\tau+1}\right\}}E_{t}\sum_{\tau=0}^{\infty}\beta^{\tau}u(Rw_{t}+y_{t}-w_{t+\tau+1})$$

Since $E_{t}u(Rw_{t}+y_{t}-w_{t+1})=u(Rw_{t}+y_{t}-w_{t+1})$, this equation can be also expressed as

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+\tau+1}\right\}}E_{t}\sum_{\tau=1}^{\infty}\beta^{\tau}u(Rw_{t+\tau}+y_{t+\tau}-w_{t+\tau+1})$$

By making the substitution $\tau=k+1$, we obtain that

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+k+2}\right\}}E_{t}\sum_{k=0}^{\infty}\beta^{k+1}u(Rw_{t+k+1}+y_{t+k+1}-w_{t+k+2})$$

$$V(w_{t},y_{t})=\max_{w_{t+1}}u(Rw_{t}+y_{t}-w_{t+1})+\max_{\left\{w_{t+k+2}\right\}}\beta E_{t}\sum_{k=0}^{\infty}\beta^{k}u(Rw_{t+k+1}+y_{t+k+1}-w_{t+k+2})$$




In [7]:
Model = namedtuple('Model', ('beta',    # Discount factor
                             'R',       # Gross interest rate
                             'gamma',   # CRRA parameter
                             'w_grid',  # Wealth grid
                             'y_grid',  # Labor income grid
                             'Q'))      # Labor income transition matrix

In [8]:
def create_consumption_model(R=1.01,         # Gross interest rate
                             beta=0.98,      # Discount factor
                             gamma=2.5,      # CRRA parameter
                             w_min=0.01,     # Min wealth
                             w_max=5.0,      # Max wealth
                             w_size=150,     # Size of the wealth grid
                             rho=0.9, nu=0.1,   # Income parameters
                             y_size=100):    # Size of the income grid

    w_grid = np.linspace(w_min, w_max, w_size)  # Create the wealth grid
    mc = qe.tauchen(rho, nu, n=y_size)          # 'qe.tauchen' computes a Markov chain associated with a discretized version of the linear Gaussian AR(1) process
                                                #    - rho is the autocorrelation coefficient
                                                #    - nu is the standard deviation of the random process
    y_grid, Q = np.exp(mc.state_values), mc.P   # - 'y_grid' is the grid of transformed income values
                                                # - 'Q' is the transition matrix describing transition probabilities between discrete income values ​​in the stochastic process

    return Model(beta=beta, R=R, gamma=gamma, w_grid=w_grid, y_grid=y_grid, Q=Q)

In [10]:
@njit
def B(i, j, ip, v, model):

    β, R, γ, w_grid, y_grid, Q = model          # Unpacked model parameters
    w, y, wp = w_grid[i], y_grid[j], w_grid[ip] # We use:
                                                #  - index i for access the value of current wealth within the wealth grid
                                                #  - index j for access the value of income within the income grid
                                                #  - index ip foe acces the value of next period wealth within the wealth grid
    c = R * w + y - wp
    if c > 0:
        return c**(1 - γ) / (1 - γ) + β * np.dot(v[ip, :], Q[j, :])
    return - np.inf